In [184]:
import pandas as pd
import whoscored_data as whoscored
df = whoscored.clean_whoscored_data(data_csv='../data/whoscored/whoscored_10000_matches.csv', tournment_id=7)

all matchs: (9661, 27)
all detailed matchs: (761, 27)


In [185]:
import sofifa_data as sofifa
sofifa_df = sofifa.clean_sofifa_data(data_csv='../data/sofifa/sofifia-english-league-championship.csv')

Merge whoscroed data with fifa data
with whoscored_team_id

In [186]:
import datetime
sofifa_df.whoscored_team_id
sofifa_df.loc[(sofifa_df.whoscored_team_id == 170) & (sofifa_df.last_update_datetime<'2016-08-05')].sort_values(by='last_update_datetime', ascending=0).head(1)

,clubs,overall,attack,midfield,defence,starting_avg_age,whole_team_avg_age,build_up_play_speed,build_up_play_dribbling,build_up_play_passing,...,chance_creation_positioning,defence_pressure,defence_aggression,defence_team_width,defence_defender_line,sofifa_team_id,transfer_budget_total,fifa_edition,last_update_datetime,whoscored_team_id
2201,Fulham,70,71,70,69,24.3,23.8,39,41,57,...,Organised,31,37,39,Cover,144,1000000.0,FIFA 16,2016-08-04,170


In [187]:
sofifa_df.head(1)

,clubs,overall,attack,midfield,defence,starting_avg_age,whole_team_avg_age,build_up_play_speed,build_up_play_dribbling,build_up_play_passing,...,chance_creation_positioning,defence_pressure,defence_aggression,defence_team_width,defence_defender_line,sofifa_team_id,transfer_budget_total,fifa_edition,last_update_datetime,whoscored_team_id
0,Nottingham Forest,69,69,69,68,25.7,25.6,69,34,57,...,Organised,39,47,44,Cover,14,450000.0,FIFA 15,2014-08-29,174


In [188]:
fifa_features=['overall', 'attack', 'midfield', 'defence', 'starting_avg_age',
       'whole_team_avg_age', 'build_up_play_speed', 'build_up_play_dribbling',
       'build_up_play_passing', 'build_up_play_positioning',
       'chance_creation_passing', 'chance_creation_crossing',
       'chance_creation_shooting', 'chance_creation_positioning',
       'defence_pressure', 'defence_aggression', 'defence_team_width',
       'defence_defender_line', 'sofifa_team_id', 'transfer_budget_total']

def get_fifa_team_info(team_id, match_date):  
    team_info = sofifa_df.loc[(sofifa_df.whoscored_team_id == team_id) & (sofifa_df.last_update_datetime<match_date)].sort_values(by='last_update_datetime', ascending=0).head(1)
    return team_info.iloc[0]    

In [189]:
for index, row in df.iterrows():
    whoscored_team_id = row.home_team_id
    match_date = row.date
    
    home_team_info = get_fifa_team_info(whoscored_team_id, match_date)
    for i, feature in enumerate(fifa_features):
        home_feature = ''.join(['home_',feature])        
        df.loc[index, home_feature] = home_team_info[feature]              
        
    away_team_info = get_fifa_team_info(whoscored_team_id, match_date)
    for i, feature in enumerate(fifa_features):
        home_feature = ''.join(['away_',feature])        
        df.loc[index, home_feature] = home_team_info[feature]                  
    
df.head()

,match_report,home_team,away_team,half_time,full_time,kick_off,date,tournament,country,home_shots,...,away_chance_creation_passing,away_chance_creation_crossing,away_chance_creation_shooting,away_chance_creation_positioning,away_defence_pressure,away_defence_aggression,away_defence_team_width,away_defence_defender_line,away_sofifa_team_id,away_transfer_budget_total
7535,Match Report,Fulham,Newcastle,1 : 0,1 : 0,19:45,2016-08-05,Championship - 2016/2017,England,5,...,39.0,46.0,49.0,Organised,31.0,37.0,39.0,Cover,144.0,1000000.0
1799,Match Report,Birmingham,Cardiff,0 : 0,0 : 0,15:00,2016-08-06,Championship - 2016/2017,England,14,...,68.0,63.0,64.0,Organised,42.0,47.0,54.0,Cover,88.0,300000.0
788,Match Report,Blackburn,Norwich,0 : 3,1 : 4,15:00,2016-08-06,Championship - 2016/2017,England,5,...,59.0,48.0,38.0,Free Form,50.0,60.0,57.0,Cover,3.0,630000.0
5650,Match Report,Bristol City,Wigan,0 : 1,2 : 1,15:00,2016-08-06,Championship - 2016/2017,England,24,...,62.0,62.0,53.0,Organised,48.0,61.0,54.0,Cover,1919.0,400000.0
5907,Match Report,Derby,Brighton,0 : 0,0 : 0,15:00,2016-08-06,Championship - 2016/2017,England,12,...,47.0,53.0,55.0,Organised,41.0,42.0,57.0,Cover,91.0,1000000.0


# 3. Training

In [190]:
whoscored_features=['team','shots','shots_on_target',
       'pass_success', 
       'aerial_duel_success', 
       'dribbles_won', 'tackles',
       'possession', 
       'team_id', 'half_time_goal',
       'full_time_goal']

fifa_features=['overall', 'attack', 'midfield', 'defence', 'starting_avg_age',
       'whole_team_avg_age', 'build_up_play_speed', 'build_up_play_dribbling',
       'build_up_play_passing', 
       'chance_creation_passing', 'chance_creation_crossing',
       'chance_creation_shooting', 
       'defence_pressure', 'defence_aggression', 'defence_team_width',
       'transfer_budget_total']

home_input_columns=[]
away_input_columns=[]

for index, feature in enumerate(whoscored_features):
    home_feature = ''.join(['home_',feature])
    away_feature = ''.join(['away_',feature])
    home_input_columns.append(home_feature)
    away_input_columns.append(away_feature)

for index, feature in enumerate(fifa_features):
    home_feature = ''.join(['home_',feature])
    away_feature = ''.join(['away_',feature])
    home_input_columns.append(home_feature)
    away_input_columns.append(away_feature)

#TODO don't take mean for fifa data (minor)  
    
home_team_info=df[home_input_columns].groupby('home_team_id').mean()
away_team_info=df[away_input_columns].groupby('away_team_id').mean()
home_team_info.head()

,home_shots,home_shots_on_target,home_pass_success,home_aerial_duel_success,home_dribbles_won,home_tackles,home_possession,home_half_time_goal,home_full_time_goal,home_overall,...,home_build_up_play_speed,home_build_up_play_dribbling,home_build_up_play_passing,home_chance_creation_passing,home_chance_creation_crossing,home_chance_creation_shooting,home_defence_pressure,home_defence_aggression,home_defence_team_width,home_transfer_budget_total
home_team_id,,,,,,,,,,,,,,,,,,,,,
19,10.916667,3.666667,0.741667,0.455000,8.500000,19.333333,0.485833,0.416667,1.250000,70.166667,...,59.500000,37.666667,56.000000,48.166667,49.250000,61.000000,40.083333,61.166667,59.250000,5.750000e+05
20,14.818182,4.363636,0.773636,0.510909,8.454545,15.090909,0.530909,0.272727,0.727273,71.727273,...,43.818182,46.181818,39.363636,47.090909,53.181818,55.000000,41.000000,42.000000,62.454545,9.454545e+05
23,15.600000,5.200000,0.790000,0.493000,6.600000,15.800000,0.595000,0.900000,2.200000,76.000000,...,58.000000,53.000000,54.900000,47.400000,53.400000,46.300000,41.000000,39.000000,51.000000,3.150000e+06
24,12.750000,4.750000,0.735000,0.567500,9.166667,18.083333,0.480000,0.666667,1.583333,74.416667,...,62.000000,61.666667,46.500000,61.250000,52.000000,45.250000,38.000000,47.000000,55.666667,2.491667e+06
25,16.363636,4.363636,0.793636,0.504545,6.909091,13.363636,0.584545,0.454545,1.454545,72.090909,...,53.363636,37.000000,48.090909,63.818182,60.363636,56.181818,50.090909,69.363636,45.636364,6.772727e+05


In [191]:
away_team_info.head()

,away_shots,away_shots_on_target,away_pass_success,away_aerial_duel_success,away_dribbles_won,away_tackles,away_possession,away_half_time_goal,away_full_time_goal,away_overall,...,away_build_up_play_speed,away_build_up_play_dribbling,away_build_up_play_passing,away_chance_creation_passing,away_chance_creation_crossing,away_chance_creation_shooting,away_defence_pressure,away_defence_aggression,away_defence_team_width,away_transfer_budget_total
away_team_id,,,,,,,,,,,,,,,,,,,,,
19,12.818182,3.454545,0.757273,0.479091,10.454545,16.909091,0.530909,0.454545,1.454545,71.000000,...,57.454545,45.909091,54.909091,55.636364,57.272727,54.454545,41.818182,49.272727,49.363636,1.009091e+06
20,11.000000,3.818182,0.744545,0.502727,7.454545,15.363636,0.480909,0.454545,1.090909,68.727273,...,49.818182,47.000000,51.363636,53.090909,54.363636,49.545455,46.272727,50.727273,50.181818,5.700000e+05
23,13.083333,5.000000,0.756667,0.529167,5.666667,17.583333,0.494167,0.916667,1.833333,69.666667,...,58.083333,42.666667,56.166667,52.916667,56.083333,51.833333,41.833333,51.333333,52.833333,8.150000e+05
24,11.416667,2.916667,0.675833,0.555833,8.416667,15.500000,0.473333,0.333333,0.583333,70.000000,...,55.916667,42.500000,57.916667,58.083333,58.500000,55.000000,45.916667,52.416667,55.083333,6.141667e+05
25,10.818182,4.272727,0.766364,0.476364,6.272727,18.727273,0.532727,0.454545,0.909091,69.636364,...,47.272727,44.727273,52.909091,51.727273,53.545455,49.090909,42.000000,49.000000,51.545455,7.390909e+05


## 3.1. prepare training set and test set

In [194]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

train_columns=home_input_columns + away_input_columns
remove_columns=['team','team_id', 'half_time_goal', 'full_time_goal']

for index, column in enumerate(remove_columns):    
    train_columns.remove(''.join(['home_',column]))    
    train_columns.remove(''.join(['away_',column]))

del remove_columns

target_column='full_time_goal_diff'
team_id_columns=['home_team_id','away_team_id']

#train_df, test_df = train_test_split(df[train_columns + [target_column] + team_id_columns], test_size = 0.2)
train_size=int(df.shape[0]*0.8)

train_df = df[train_columns + [target_column] + team_id_columns][:train_size]
test_df = df[train_columns + [target_column] + team_id_columns][train_size:]
train_x_df = train_df[train_columns]
train_y_df = train_df[target_column]
test_x_df = test_df[train_columns]
test_y_df = test_df[target_column]


## 3.2. linear regression

In [196]:
import numpy as np

ols_regr = LinearRegression()
ols_regr.fit(train_x_df, train_y_df)
print('Coefficients: \n', ols_regr.coef_)
print("Mean squared error: %.2f" % np.mean((ols_regr.predict(test_x_df) - test_y_df) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % ols_regr.score(test_x_df, test_y_df))
print('Mean absolute error: %.2f' % mean_absolute_error(test_y_df,ols_regr.predict(test_x_df)))
regr = ols_regr

Coefficients: 
 [ -7.89867959e-02   3.93373078e-01   1.53400100e+00   2.65356394e+01
   3.33300105e-02  -3.41624072e-03  -2.24720010e+00   2.44520702e-02
  -4.29948334e-02  -2.89934399e-02   6.36003414e-02   3.36242991e-02
  -2.15716998e-02  -1.08270044e-02  -1.46550454e-02  -4.53705845e-03
  -1.85368230e-03   2.25232935e-03  -2.73722761e-03   4.09485248e-03
  -5.12276413e-03   1.33563552e-03   2.33347715e-07   2.97887816e-02
  -3.23082096e-01  -2.72615199e+00   2.51758706e+01  -9.24003689e-03
  -2.15185049e-02   2.24720010e+00   2.44520702e-02  -4.29948334e-02
  -2.89934399e-02   6.36003414e-02   3.36242991e-02  -2.15716998e-02
  -1.08270044e-02  -1.46550454e-02  -4.53705845e-03  -1.85368230e-03
   2.25232935e-03  -2.73722762e-03   4.09485248e-03  -5.12276414e-03
   1.33563553e-03  -1.94129094e-07]
Mean squared error: 2.32
Variance score: 0.23
Mean absolute error: 1.21


## 3.3. Huber Regressor

In [197]:
from sklearn.linear_model import HuberRegressor
hregr = HuberRegressor()
hregr.fit(train_x_df, train_y_df)

print('Coefficients: \n', hregr.coef_)
print("Mean squared error: %.2f" % np.mean((hregr.predict(test_x_df) - test_y_df) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % hregr.score(test_x_df, test_y_df))
print('Mean absolute error: %.2f' % mean_absolute_error(test_y_df,hregr.predict(test_x_df)))


Coefficients: 
 [  3.11970520e-12   2.65247828e-12   1.22202777e-13   1.33864230e-13
   2.65049313e-12   3.44634353e-12   6.44583016e-14   1.32982870e-11
   1.33164345e-11   1.31139467e-11   1.34702017e-11   5.11500685e-12
   4.66659276e-12   8.16729475e-12   5.50696733e-12   1.19499032e-11
   1.09009867e-11   1.07484799e-11   9.62095431e-12   8.86576539e-12
   1.02794534e-11   1.17406314e-11   1.39197148e-07   1.07638421e-12
  -1.02150174e-12   1.32875301e-13   5.45849585e-14   7.89386304e-13
   2.32384866e-12   1.23901612e-13   1.32982870e-11   1.33164345e-11
   1.31139467e-11   1.34702017e-11   5.11500685e-12   4.66659276e-12
   8.16729475e-12   5.50696733e-12   1.19499032e-11   1.09009867e-11
   1.07484799e-11   9.62095431e-12   8.86576539e-12   1.02794534e-11
   1.17406314e-11   1.39197148e-07]
Mean squared error: 2.92
Variance score: 0.03
Mean absolute error: 1.29


## 3.4. Lasso

In [198]:
from sklearn.linear_model import Lasso
clf = Lasso(alpha=0.1)
clf.fit(train_x_df, train_y_df)

print('Coefficients: \n', clf.coef_)
print("Mean squared error: %.2f" % np.mean((clf.predict(test_x_df) - test_y_df) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % clf.score(test_x_df, test_y_df))
print('Mean absolute error: %.2f' % mean_absolute_error(test_y_df,clf.predict(test_x_df)))


Coefficients: 
 [ -7.25241356e-02   3.61165856e-01  -0.00000000e+00   0.00000000e+00
   2.63433038e-02   9.73756389e-03  -0.00000000e+00   0.00000000e+00
  -0.00000000e+00  -0.00000000e+00   2.49901229e-02   0.00000000e+00
   0.00000000e+00  -1.40331785e-02  -1.67491685e-02   5.18688706e-04
  -2.57124651e-03   4.63732429e-03  -5.85705283e-03  -0.00000000e+00
  -2.45152426e-04   9.97294443e-03   8.75957901e-08   1.93867998e-02
  -3.11056820e-01   0.00000000e+00  -0.00000000e+00  -5.16782802e-03
  -2.98693036e-02   0.00000000e+00   0.00000000e+00  -0.00000000e+00
  -0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
  -7.44815901e-03  -6.58304412e-03   2.69670065e-04  -0.00000000e+00
   2.54548804e-04  -4.35072491e-03  -0.00000000e+00  -3.59293780e-03
   5.20434939e-04  -5.99533881e-08]
Mean squared error: 2.14
Variance score: 0.29
Mean absolute error: 1.21


## 3.5. Elastic Net

In [199]:
from sklearn.linear_model import ElasticNet

enf = ElasticNet(alpha=0.1)
enf.fit(train_x_df, train_y_df)

print('Coefficients: \n', enf.coef_)
print("Mean squared error: %.2f" % np.mean((enf.predict(test_x_df) - test_y_df) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % enf.score(test_x_df, test_y_df))
print('Mean absolute error: %.2f' % mean_absolute_error(test_y_df,enf.predict(test_x_df)))


Coefficients: 
 [ -7.99146193e-02   3.77844799e-01  -0.00000000e+00   0.00000000e+00
   2.99870281e-02   9.53955274e-03  -0.00000000e+00   0.00000000e+00
  -1.85547072e-02  -7.61059936e-04   3.91246107e-02   3.90410726e-03
   0.00000000e+00  -1.35343781e-02  -1.58482365e-02  -0.00000000e+00
  -3.65912014e-03   4.70283532e-03  -5.61593517e-03  -0.00000000e+00
  -1.34338151e-03   6.64308159e-03   9.05662212e-08   2.48236961e-02
  -3.24716251e-01   0.00000000e+00  -0.00000000e+00  -7.66480280e-03
  -3.13530219e-02   0.00000000e+00   0.00000000e+00  -1.93208308e-02
  -4.47858497e-03   2.98461755e-02   2.89502567e-03   0.00000000e+00
  -8.25314297e-03  -1.00450930e-02  -0.00000000e+00  -0.00000000e+00
   1.70045355e-03  -6.01604354e-03  -0.00000000e+00  -3.79354989e-03
   3.07180288e-03  -5.54235102e-08]
Mean squared error: 2.16
Variance score: 0.28
Mean absolute error: 1.22


# 4. Prediction
## 4.1. Whoscored Team ID INFO

In [200]:
home_team_id_map=df.groupby(['home_team_id','home_team']).count().reset_index()[['home_team_id','home_team']]
away_team_id_map=df.groupby(['away_team_id','away_team']).count().reset_index()[['away_team_id','away_team']]


def team_id_to_name(team_id):
    away_team_id_name=away_team_id_map.loc[away_team_id_map['away_team_id']==team_id]    
    if away_team_id_name.size > 0:
        return away_team_id_name['away_team'].values[0]
    
    home_team_id_name=home_team_id_map.loc[home_team_id_map['home_team_id']==team_id]
    if home_team_id_name.size > 0:
        return home_team_id_name['home_team'].values[0]
    return None

for team_id in range(0, 1000):
    team_name = team_id_to_name(team_id)
    if team_name is not None:
        print(str(team_id),':',team_id_to_name(team_id))

19 : Leeds
20 : Derby
23 : Newcastle
24 : Aston Villa
25 : Sheff Wed
94 : Reading
142 : Barnsley
157 : Birmingham
158 : Blackburn
161 : Wolves
165 : Ipswich
166 : Huddersfield
168 : Norwich
170 : Fulham
171 : QPR
174 : Nottingham Forest
181 : Preston
182 : Bristol City
188 : Cardiff
189 : Brentford
194 : Wigan
210 : Rotherham
211 : Brighton


## 4.3. Make prediction with all test data set

In [201]:
def predict_diff(home_team_id, away_team_id):
    home_team_info_1=home_team_info.reset_index()
    home_team_features=home_team_info_1.loc[home_team_info_1['home_team_id']==home_team_id]
    del home_team_info_1
    
    away_team_info_1=away_team_info.reset_index()        
    away_team_features=away_team_info_1.loc[away_team_info_1['away_team_id']==away_team_id]
    del away_team_info_1
    
    features = pd.concat([home_team_features.reset_index(), away_team_features.reset_index()], axis=1)[train_columns]
    features['total_possession']=features['home_possession']+features['away_possession']
    features['home_possession']=features['home_possession']/features['total_possession']
    features['away_possession']=features['away_possession']/features['total_possession']
    features.drop('total_possession',axis=1,inplace=True)    
    return {'ols_regr':ols_regr.predict(features),'hregr':hregr.predict(features),'clf':clf.predict(features),'enf':enf.predict(features)}

In [202]:
test_df_predict = test_df[['home_team_id','away_team_id','full_time_goal_diff']]

In [204]:
for index, row in test_df_predict.iterrows():            
    predictions=predict_diff(row['home_team_id'],row['away_team_id'])
    #print(row['full_time_goal_diff'],":",predicted_diff)    
    test_df_predict.loc[index, 'prediction(Linear Regression)'] = predictions['ols_regr']
    test_df_predict.loc[index, 'prediction(Huber Regressor)'] = predictions['hregr']
    test_df_predict.loc[index, 'prediction(Lasso)'] = predictions['clf']
    test_df_predict.loc[index, 'prediction(Elastic Net)'] = predictions['enf']

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [205]:
test_df_predict

,home_team_id,away_team_id,full_time_goal_diff,prediction(Linear Regression),prediction(Huber Regressor),prediction(Lasso),prediction(Elastic Net)
3645,157,211,-1,1.499850,0.147528,0.378009,0.360406
178,210,94,-1,-1.955271,0.135021,0.061988,0.110655
7550,210,1786,-1,-1.414193,0.137752,-0.011359,0.032066
2179,157,142,-3,0.065075,0.104250,0.085436,0.060966
5340,194,94,-3,0.185270,0.166024,0.237695,0.268839
3315,158,94,-1,0.361145,0.168713,0.545716,0.569216
1248,25,171,1,0.098366,0.176679,0.169954,0.084807
8333,161,171,-1,-0.138203,0.210213,0.539756,0.557962
2978,158,166,0,-2.123739,0.209956,0.171643,0.188450
1518,161,20,-1,-0.084440,0.207151,0.472492,0.504614


In [223]:
model1=test_df_predict['full_time_goal_diff']-test_df_predict['prediction(Linear Regression)']
print(model1.round().abs().mean())
model2=test_df_predict['full_time_goal_diff']-test_df_predict['prediction(Huber Regressor)']
print(model2.round().abs().mean())
model3=test_df_predict['full_time_goal_diff']-test_df_predict['prediction(Lasso)']
print(model3.round().abs().mean())
model4=test_df_predict['full_time_goal_diff']-test_df_predict['prediction(Elastic Net)']
print(model4.round().abs().mean())
model0=test_df_predict['full_time_goal_diff']
print(model0.round().abs().mean())

1.49090909091
1.29090909091
1.27272727273
1.29090909091
1.29090909091
